In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
#from 


In [2]:
# import data 

df = pd.read_csv('telecom_customer_churn_cleaned.csv')
df.head()


,Customer_ID,Gender,Age,Married,Number_of_Dependents,City,Zip_Code,Latitude,Longitude,Number_of_Referrals,...,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Customer_Status,Churn_Category,Churn_Reason,Num_Internet_Features,Has_Multiple_Lines,Churn_status
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,...,0.00,0,381.51,974.81,Stayed,NaN,NaN,5.0,1.0,0
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,...,38.33,10,96.21,610.28,Stayed,NaN,NaN,3.0,2.0,0
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,...,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices,3.0,1.0,1
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,...,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction,6.0,1.0,1
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,...,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability,4.0,1.0,1


In [3]:
df.columns

Index(['Customer_ID', 'Gender', 'Age', 'Married', 'Number_of_Dependents',
       'City', 'Zip_Code', 'Latitude', 'Longitude', 'Number_of_Referrals',
       'Tenure_in_Months', 'Offer', 'Avg_Monthly_Long_Distance_Charges',
       'Internet_Type', 'Avg_Monthly_GB_Download', 'Contract',
       'Paperless_Billing', 'Payment_Method', 'Monthly_Charge',
       'Total_Charges', 'Total_Refunds', 'Total_Extra_Data_Charges',
       'Total_Long_Distance_Charges', 'Total_Revenue', 'Customer_Status',
       'Churn_Category', 'Churn_Reason', 'Num_Internet_Features',
       'Has_Multiple_Lines', 'Churn_status'],
      dtype='object')

In [4]:
# removing revenue and monthly charge because it is highly corrleated with tenure (Checked in the  last cell)
# Removing geography based features for now (Not part of the scope)
# Churn reason and Churn Category are removed because very sparsely populated 

to_remove = ['Customer_ID','City', 'Zip_Code', 'Latitude', 'Longitude' ,'Total_Charges', 'Total_Refunds', 'Total_Extra_Data_Charges',
       'Total_Long_Distance_Charges','Churn_Category', 'Churn_Reason' ,'Customer_Status' , 'Total_Revenue', 'Monthly_Charge']

# filtering out the columns we will use for the analysis

columns_list = [col for col in df.columns if col not in to_remove]

df_1 = df[columns_list]
df_1.head()

df_1.columns


Index(['Gender', 'Age', 'Married', 'Number_of_Dependents',
       'Number_of_Referrals', 'Tenure_in_Months', 'Offer',
       'Avg_Monthly_Long_Distance_Charges', 'Internet_Type',
       'Avg_Monthly_GB_Download', 'Contract', 'Paperless_Billing',
       'Payment_Method', 'Num_Internet_Features', 'Has_Multiple_Lines',
       'Churn_status'],
      dtype='object')

In [5]:
# This is the template we expect to get from the user ( Will need this while making GUI )

df_1.to_csv('data_input_test.csv', index=False)

In [6]:
# Numerical columns 

# All the numerical variables in the dataset
numerical_columns = df_1.select_dtypes(include=['int64', 'float64']).columns.tolist()

# remove Churn_status and Tenure_in_Months

numerical_columns.remove('Churn_status')
numerical_columns.remove('Tenure_in_Months')

print("The numerical columns are - \n " , numerical_columns)

# Feature scaling the numerical variables

#scaler = StandardScaler()

#df_1[numerical_columns] = scaler.fit_transform(df_1[numerical_columns])


The numerical columns are - 
  ['Age', 'Number_of_Dependents', 'Number_of_Referrals', 'Avg_Monthly_Long_Distance_Charges', 'Avg_Monthly_GB_Download', 'Num_Internet_Features', 'Has_Multiple_Lines']


In [7]:
# All the categorical variables in the dataset

categorical_columns = df_1.select_dtypes(include=['object']).columns.tolist()

print("Categorical columns are: ", categorical_columns)

Categorical columns are:  ['Gender', 'Married', 'Offer', 'Internet_Type', 'Contract', 'Paperless_Billing', 'Payment_Method']


In [8]:
# encoding categorical variables with 2 categories

two_cat = []

# finding the columns with 2 categories

for i in categorical_columns:
    #print(i)
    #print(df_1[i].nunique())
    if df_1[i].nunique() == 2:
        two_cat.append(i)

print("The variables with 2 categories" , two_cat)



The variables with 2 categories ['Gender', 'Married', 'Paperless_Billing']


In [9]:
# Encoding the categorical variables with 2 categories

gen_conditions = [df_1['Gender'] == 'Male',df_1['Gender'] == 'Female']
married_conditions = [df_1['Married'] == 'Yes',df_1['Married'] == 'No']
paper_conditions = [df_1['Paperless_Billing'] == 'Yes',df_1['Paperless_Billing'] == 'No']



df_2 = df_1.assign(
    gender_e = np.select(gen_conditions, [1, 0], default=0),
    married_e = np.select(married_conditions, [1, 0], default=0),
    #phone_e = np.select(phone_conditions, [1, 0], default=0),
    #internet_e = np.select(internet_conditions, [1, 0], default=0),
    paper_e = np.select(paper_conditions, [1, 0], default=0)
)

print("Encoding completed")

Encoding completed


In [10]:
# Encoding the categorical variables with more than 2 categories

multiple_cat = []

# finding the columns with 3 categories

for i in categorical_columns:
    #print(i)
    #print(df_1[i].nunique())
    if df_2[i].nunique() >= 3:
        multiple_cat.append(i)

print("Variables with more than 2 categories" , multiple_cat)

# remove customer_id 

Variables with more than 2 categories ['Offer', 'Internet_Type', 'Contract', 'Payment_Method']


In [11]:
# Encoding offer 

"""
We are NOT one hot encoding offer because one hot encoded offer (offer A , B , C) 
had very poor residual dsitr i.e Offer A : 0 and Offer A : 1 had high negative residual 
(checked in the residual plot after fitting the model) , so simplying it to offer and no offer (0 and 1)

"""

conditions = [
    df_2['Offer'].isin(['Offer A', 'Offer B', 'Offer C', 'Offer D', 'Offer E'])
]

choices = [1]  # Value when condition is True

df_2['offer_e'] = np.select(conditions, choices, default=0)

# drop offer from the dataframe and multiple cat list

df_2.drop(columns = 'Offer', inplace = True)

multiple_cat.remove('Offer')

print("Offer encoding done")


Offer encoding done


In [12]:
# Encoding "2 year contract" & "Payment_Method" - Similar reasons as "offer"

df_2['Contract_e'] = df_2['Contract'].isin(['Two Year','One Year']).astype(int)

df_2['Payment_Method_e'] = df_2['Payment_Method'].isin(['Credit Card']).astype(int)

multiple_cat.remove('Contract')

multiple_cat.remove('Payment_Method')

df_2.drop(columns=['Contract','Payment_Method'] , inplace = True)


In [13]:

# One-hot encoding columns with more than 2 categories (Except the ones we handled otherwise)

df_3 = pd.get_dummies(df_2, columns=multiple_cat, drop_first=True , dtype='int')

df_3.head()

df_3.columns

Index(['Gender', 'Age', 'Married', 'Number_of_Dependents',
       'Number_of_Referrals', 'Tenure_in_Months',
       'Avg_Monthly_Long_Distance_Charges', 'Avg_Monthly_GB_Download',
       'Paperless_Billing', 'Num_Internet_Features', 'Has_Multiple_Lines',
       'Churn_status', 'gender_e', 'married_e', 'paper_e', 'offer_e',
       'Contract_e', 'Payment_Method_e', 'Internet_Type_DSL',
       'Internet_Type_Fiber Optic'],
      dtype='object')

In [14]:
# final column list ( Removed columns are the ones which are already encoded)

final_columns = [col for col in df_3.columns if col not in ['Gender', 'Married', 'Phone_Service', 'Internet_Service','Paperless_Billing']]

In [15]:
df_4 = df_3[final_columns]
df_4.head()

,Age,Number_of_Dependents,Number_of_Referrals,Tenure_in_Months,Avg_Monthly_Long_Distance_Charges,Avg_Monthly_GB_Download,Num_Internet_Features,Has_Multiple_Lines,Churn_status,gender_e,married_e,paper_e,offer_e,Contract_e,Payment_Method_e,Internet_Type_DSL,Internet_Type_Fiber Optic
0,37,0,2,9,42.39,16.0,5.0,1.0,0,0,1,1,0,1,1,0,0
1,46,0,0,9,10.69,10.0,3.0,2.0,0,1,0,0,0,0,1,0,0
2,50,0,0,4,33.65,30.0,3.0,1.0,1,1,0,1,1,0,0,0,1
3,78,0,1,13,27.82,4.0,6.0,1.0,1,1,1,1,1,0,0,0,1
4,75,0,3,3,7.38,11.0,4.0,1.0,1,0,1,1,0,0,1,0,1


In [16]:
df_4.columns

Index(['Age', 'Number_of_Dependents', 'Number_of_Referrals',
       'Tenure_in_Months', 'Avg_Monthly_Long_Distance_Charges',
       'Avg_Monthly_GB_Download', 'Num_Internet_Features',
       'Has_Multiple_Lines', 'Churn_status', 'gender_e', 'married_e',
       'paper_e', 'offer_e', 'Contract_e', 'Payment_Method_e',
       'Internet_Type_DSL', 'Internet_Type_Fiber Optic'],
      dtype='object')

In [17]:
# Dump the data to be used to fit the model

df_4.to_csv('telecom_customer_churn_cleaned_encoded.csv', index=False)

In [18]:
### Correlation check 


# Compute correlation matrix
corr_matrix = df_4.corr().abs()

# Loop through upper triangle and print highly correlated pairs
threshold = 0.5 # You can change this to 0.8, 0.95, etc.
printed = set()

for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)):
        col1 = corr_matrix.columns[i]
        col2 = corr_matrix.columns[j]
        corr_val = corr_matrix.iloc[i, j]
        if corr_val > threshold:
            print(f"{col1} ↔ {col2} | correlation = {corr_val:.3f}")


Number_of_Referrals ↔ married_e | correlation = 0.673
Tenure_in_Months ↔ Contract_e | correlation = 0.628
